In [1]:
import pandas as pd

data = {
    '搜尋詞': ['淋浴水龍頭']*10,
    'Rank': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'tf-idf': [
        'JAVA 雙頭淋浴水龍頭 TLS414, 1組', 
        'MIRINAE 壁掛式淋浴水龍頭 MF-102, 1組', 
        'American Standard 美標 淋浴水龍頭 FB2689, 1個', 
        'DAELIM BATH 洗臉台淋浴水龍頭 DL-B1615, 1組', 
        'HANSSEM 淋浴方形水龍頭 HI-9030', 
        '高級水龍頭 黑色, 1個', 
        '高級水龍頭 藍色, 1個', 
        'YJJ 水龍頭過濾器3.0型水槽旋轉水龍頭龍頭套裝, 1組', 
        '水龍頭全浴美容洗髮水台水龍頭TPF-0105, 1套', 
        '我們的水早晨壁掛式單桿淋浴浴缸水龍頭, 1個'],
    'ner_relevancy_1': [0] * 10,
    'semantic_model': [
        '淋浴水龍頭, 1入',
        '淋浴水龍頭組 黑色, 1個', 
        '趨勢淋浴豪華浴室水龍頭鍍鉻光澤 RA-600, 1個', 
        '藍水浴室壁掛式淋浴龍頭 2-600, 1個', 
        'Jeilbas 鍍鉻鍍金浴室淋浴龍頭套組, 1套', 
        '水龍頭龍頭兩孔頂級水槽管道淋浴龍頭EINS-3033, 1個', 
        '水壓上升節水淋浴噴頭銀色, 1個', 
        '淋浴龍頭 T-02, 1個', 
        '壁掛式浴室淋浴龍頭金屬 OV3-600, 1個', 
        '淋浴龍頭 GK-102, 1個'],
    'ner_relevancy_2': [0] * 10
}

df = pd.DataFrame(data)
df

,搜尋詞,Rank,tf-idf,ner_relevancy_1,semantic_model,ner_relevancy_2
0,淋浴水龍頭,1,"JAVA 雙頭淋浴水龍頭 TLS414, 1組",0,"淋浴水龍頭, 1入",0
1,淋浴水龍頭,2,"MIRINAE 壁掛式淋浴水龍頭 MF-102, 1組",0,"淋浴水龍頭組 黑色, 1個",0
2,淋浴水龍頭,3,"American Standard 美標 淋浴水龍頭 FB2689, 1個",0,"趨勢淋浴豪華浴室水龍頭鍍鉻光澤 RA-600, 1個",0
3,淋浴水龍頭,4,"DAELIM BATH 洗臉台淋浴水龍頭 DL-B1615, 1組",0,"藍水浴室壁掛式淋浴龍頭 2-600, 1個",0
4,淋浴水龍頭,5,HANSSEM 淋浴方形水龍頭 HI-9030,0,"Jeilbas 鍍鉻鍍金浴室淋浴龍頭套組, 1套",0
5,淋浴水龍頭,6,"高級水龍頭 黑色, 1個",0,"水龍頭龍頭兩孔頂級水槽管道淋浴龍頭EINS-3033, 1個",0
6,淋浴水龍頭,7,"高級水龍頭 藍色, 1個",0,"水壓上升節水淋浴噴頭銀色, 1個",0
7,淋浴水龍頭,8,"YJJ 水龍頭過濾器3.0型水槽旋轉水龍頭龍頭套裝, 1組",0,"淋浴龍頭 T-02, 1個",0
8,淋浴水龍頭,9,"水龍頭全浴美容洗髮水台水龍頭TPF-0105, 1套",0,"壁掛式浴室淋浴龍頭金屬 OV3-600, 1個",0
9,淋浴水龍頭,10,"我們的水早晨壁掛式單桿淋浴浴缸水龍頭, 1個",0,"淋浴龍頭 GK-102, 1個",0


In [2]:
from src import inference_api
import torch, src.config as config

# all_attribute = ['品牌', '名稱', '產品', '產品序號', '顏色', '材質', '對象與族群', '適用物體、事件與場所', 
#                      '特殊主題', '形狀', '圖案', '尺寸', '重量', '容量', '包裝組合', '功能與規格']

# set device
config.string_device = 'cuda' if torch.cuda.is_available() else 'cpu'
config.device = torch.device(config.string_device)

# load model
model, tokenizer = inference_api.load_model("clw8998/Product-Name-NER-model", device=config.device)

# Relevancy Calculation Function: Handles the comparison and calculation of relevancy scores
def ner_relevancy(df, index, all_results, check_att, margin):
    query_tags_dict = all_results[df.loc[index, '搜尋詞'].lower()]
    tfidf_tags_dict = all_results[df.loc[index, 'tf-idf'].lower()]
    semantic_tags_dict = all_results[df.loc[index, 'semantic_model'].lower()]

    query_tags_pool = set(ent[0] for ents in query_tags_dict.values() for ent in ents if ent)
    tfidf_tags_pool = set(ent[0] for ents in tfidf_tags_dict.values() for ent in ents if ent)
    semantic_tags_pool = set(ent[0] for ents in semantic_tags_dict.values() for ent in ents if ent)

    # 計算 query tags 與 tf-idf 的 tags 相關性
    if len(query_tags_pool & tfidf_tags_pool) >= len(check_att) * margin:
        df.loc[index, 'ner_relevancy_1'] = 2
    elif len(query_tags_pool & tfidf_tags_pool) >= len(check_att) * margin * 0.5:
        df.loc[index, 'ner_relevancy_1'] = 1
    else:
        df.loc[index, 'ner_relevancy_1'] = 0

    # 計算 query tags 與 semantic model 的 tags 相關性
    if len(query_tags_pool & semantic_tags_pool) >= len(check_att) * margin:
        df.loc[index, 'ner_relevancy_2'] = 2
    elif len(query_tags_pool & semantic_tags_pool) >= len(check_att) * margin * 0.5:
        df.loc[index, 'ner_relevancy_2'] = 1
    else:
        df.loc[index, 'ner_relevancy_2'] = 0


    return df

c:\Users\clw20\anaconda3\envs\cl\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
check_att = ['品牌', '產品', '顏色', '適用物體、事件與場所', '功能與規格']
all_results = inference_api.get_ner_tags(
        model, 
        tokenizer, 
        list(set(df['搜尋詞'].tolist() + df['tf-idf'].tolist() + df['semantic_model'].tolist())), 
        check_att)

100%|██████████| 4/4 [00:05<00:00,  1.37s/it]


In [4]:
for index, row in df.iterrows():
    df = ner_relevancy(df, index, all_results, check_att, 0.3)
df

,搜尋詞,Rank,tf-idf,ner_relevancy_1,semantic_model,ner_relevancy_2
0,淋浴水龍頭,1,"JAVA 雙頭淋浴水龍頭 TLS414, 1組",2,"淋浴水龍頭, 1入",2
1,淋浴水龍頭,2,"MIRINAE 壁掛式淋浴水龍頭 MF-102, 1組",2,"淋浴水龍頭組 黑色, 1個",2
2,淋浴水龍頭,3,"American Standard 美標 淋浴水龍頭 FB2689, 1個",2,"趨勢淋浴豪華浴室水龍頭鍍鉻光澤 RA-600, 1個",1
3,淋浴水龍頭,4,"DAELIM BATH 洗臉台淋浴水龍頭 DL-B1615, 1組",2,"藍水浴室壁掛式淋浴龍頭 2-600, 1個",1
4,淋浴水龍頭,5,HANSSEM 淋浴方形水龍頭 HI-9030,2,"Jeilbas 鍍鉻鍍金浴室淋浴龍頭套組, 1套",1
5,淋浴水龍頭,6,"高級水龍頭 黑色, 1個",1,"水龍頭龍頭兩孔頂級水槽管道淋浴龍頭EINS-3033, 1個",1
6,淋浴水龍頭,7,"高級水龍頭 藍色, 1個",1,"水壓上升節水淋浴噴頭銀色, 1個",1
7,淋浴水龍頭,8,"YJJ 水龍頭過濾器3.0型水槽旋轉水龍頭龍頭套裝, 1組",1,"淋浴龍頭 T-02, 1個",1
8,淋浴水龍頭,9,"水龍頭全浴美容洗髮水台水龍頭TPF-0105, 1套",1,"壁掛式浴室淋浴龍頭金屬 OV3-600, 1個",1
9,淋浴水龍頭,10,"我們的水早晨壁掛式單桿淋浴浴缸水龍頭, 1個",2,"淋浴龍頭 GK-102, 1個",1


In [5]:
# df.to_csv('M11207321-NER-relevancy.csv', index=False, encoding='utf-8-sig')